In [9]:
import openai
import PyPDF2
import tabula
import json
import pypdf
import re


openai.api_type = "azure"
openai.api_key = "18e14815c4a04777aa7da8827858dd20"
openai.api_base = "https://gtp-ner-api.openai.azure.com/"
openai.api_version = "2023-03-15-preview"

instruct = """
[CONTEXT] As an assistant you are going to act as an information extraction and a Key-Value pair recognition service
&nbsp;
[TASK]: Extract the Key-Value pairs from the following text and provide the correct values in a JSON format with the keys listed below. Please note the following guidelines:
•	Only include key-values that are present in the keywords list or that might be synonyms of the keywords list.
•	Maintain the same format as in the original text.
•	Do not repeat the same value for different keys.
•	If there is/are keys/values not present in the text leave them as ''.
•	If there are multiple key-values, extract only one and the most relevant.

the keywords list is the following:

•  In-Place NOI
•  Valuation NOI Basis
•  Stabilized NOI (Development Projects)
•  Occupancy
•  Stabilize / Non-Stabilized
•  Replacement Reserves, 
•  Adjusted NOI
•  Market Cap Rate Yr. 1
•  Discount Rate (IRR Unlevered)
•  Purchase Price
•  DCF Value
•  Replacement Cost
•  Return on Cost", 
•  Unlevered IRR
•  Unlevered Equity Multiple
•  Avg. Free and Clear Return
•  Levered IRR
•  Levered Equity Multiple
•  Avg. Cash-on-Cash Return
•  Return on Equity", 
•  Minimum Debt Service Coverage Ratio (NOI)
•  Min. Debt Yield (NOI)
•  WALT
•  Analysis Start Date
•  Hold Period
•  General Inflation
•  Market Rent PSF, 
•  Market Rent Growth
•  Renewal Probability
•  Capital Reserve - PSF
•  Rent Escalations
•  Tenant Improvements PSF - New
•  Tenant Improvements PSF - Renewal", 
•  Leasing Commissions New
•  Leasing Commission Renewal
•  Expense Reimbursement Type
•  Address
•  Land Area (Acres)
•  Land Area (SF) 
•  Parking Spaces", 
•  Gross Building Area (GBA) - Square Feet
•  Net Rentable Area (NRA) - Square Feet
•  Number of Stories
•  Year Built
•  Year Renovated
•  Block & Lot Number", 
•  Zoning
•  Elevator Service
•  Building Class
•  Current Use Type (Property Type)
•  Proposed Use Type
•  Community Type
•  Number of Units ", 
•  Average Unit Size - Square Feet
•  Unit Mix
•  Average Rent Per Unit
•  Average Rent Per SF
•  Community Amenities
•  Unit Amenities 
•  Year 1 NOI", 
•  NOI Basis - Exit Valuation NOI
•  Adjusted Year 1 NOI
•  In-Place Cap Rate
•  Stablized Cap Rate
•  Min. DSCR
•  Weighted Avg Leasing Term
•  Cash Flow Start Date", 
•  Analysis Hold
•  Expense Growth 
•  Rent Growth
•  Tenant Renewal Probability
•  TI New
•  TI Renewal
•  Site Area (Acres)
•  Site Area (SF)
•  Parking", 
•  Building Size - SF
•  Rentable Area - SF
•  Floors
•  Parcel ID
•  Property Subtype
•  Community Size",
•  NOI
•  Stabilize
•  Non-Stabilize
•  Stabilized
•  Non-Stabilized
•  Minimum Debt Service Coverage Ratio
•  Min. Debt Yield
•  In Place NOI
•  Stabilized NOI,
•  Development Projects
•  DCF
•  Net Rentable Area
•  NRA
•  Land Area
•  Area
•  Avg Rent
•  Average Rent
•  Avg. Rent PSF (NNN)
•  BLOCK / LOTS
•  Net Operating Income,
•  Market Rent
•  Market Rent Inflation
•  Capital Reserves
•  Capital Reserve
•  Term Length
•  TI (new / renew)
•  LC (new / renew)
•  Expense Reimbursements
&nbsp;
Extract the key-value pairs from the following text keeping the original key-value pairs text and then populate it with its elements in a JSON using the following format: 
{ "KEY_1": "string", "KEY_2": "string", "KEY_3": "string", "KEY_4": "string", "KEY_5": "string"}:
"""

examples = """
&nbsp;
[KEY-VALUES EXAMPLES]
[EXAMPLE 1]:
[TEXT]:
'THE OFFERING\n
As exclusive agent, Cushman & Wakefield’s National \n
Industrial Advisory Group is pleased to offer a 100% \n
fee interest in 1 Cory Road for sale. Located less than \n
a mile from downtown Morristown, this 296,000 \n
SF building on +/-18 acres represents the rare \n
opportunity to an acquire infill industrial building \nwith an opportunistic investment profile.  \n
The property features 25 loading positions, 20’ \n
clear heights, and is fully leased to four tenants at \n
an average in-place rent of $4.44 PSF, a fraction \n
of today’s $10.75 PSF market rate. The buildings \n
largest tenant (50% of GLA) rolls within 20 months, \n
presenting the near-term ability to mark-to-market \n
or reposition the building for high throughput use. \n
The property includes +/- 6.5 acres for parking \n
which is currently underutilized and under-rented.  Situated less than five minutes from both Exit 36 on \n
I-287 and Exit 1B on Route 24, the Property offers \n
proximate access to NYC, the Port of NY/NJ, and \n
the 475,000 residents within a ten-mile radius of \n
Morristown.  The Property is walkable to downtown \n
Morristown, a mass-transit oriented suburban CBD, \n
home to C-suite executives and a rapidly growing \n
labor pool.\n
1 Cory Road offers investors with multiple paths \n
to value creation via significant mark-to-market, \n
repositioning for high throughput usage, and \n
activating +/-6.5 acres of underutilized parking.  \n
NOI is projected to nearly double from $1.77M to \n
$3.3M in just three years.AREA: \n
296,000 SFNOI:  \n
$1.77MAVG RENT: \n
$4.44 PSFPROPERTY SNAPSHOT\n
FINANCIAL SUMMARYWALT:  \n
3.42 YEARS\n
Year 1 Year 2 Year 3\n 
Avg. Rent PSF (NNN): $4.77 $5.82 $8.61\n 
Net Operating Income $1,765,064 $2,220,421 $3,332,937 90 % NOI  \n
GROWTH\n
02 1 CORY ROAD, MORRISTOWN, NJ'

&nbsp;
[CORRECT VALUES FOR JSON KEYS]:
{
"Land Area (SF)": "296,000 SF",
"Avg Rent": "$4.44 PSF",
"Avg. Rent PSF (NNN)": "$4.77",
"Avg. Rent PSF (NNN)": "$5.82",
"Avg. Rent PSF (NNN)": "$8.61",
"Net Operating Income": "$1,765,064",
"Net Operating Income": "$2,220,421",
"Net Operating Income": "$3,332,937",
"NOI": "$1.77M",
"WALT": "3.42 YEARS"
}
&nbsp;
[EXAMPLE 2]:
[TEXT]:
'MARTIN LUTHER KING AVENUE\n
6.5 ACRES \n
UNDERUTILIZED PARKING\n
CORY ROADACCESS  \n
Two way \n
ingress/egressACCESS  \n
Two way \n
ingress/egress\n
ACCESS  \n
Two way \n
ingress/egress\n
Car Link 93,440 SF 3/31/28 LXD\n
All Modes Transport \n
151,000 SF 7/31/23 LXD\n
Preferred Plastics & Packaging 43,560 SF 3/31/26 LXDLAND AREA  \n
17.89 AcresYEAR BUILT  \n
1986\n
COLUMN SPACING  \n
40x25  BUILDING AREA \n
296,000 SF\n
PARKING  \n
208 car, 80 Truck/Bus on +6.5 acres\n
CLEAR HEIGHT  \n
20’ROOF  \n
Firestone (2005)SPRINKLERS  \n
Wet System\n
LOADING POSITIONS  \n
25 (17 Dock High, 8 Drive-In)  \n
TRUCK COURT DEPTH: +/- 120’\n
ZONING  \n
Block 10401 Lot \n
2 (I-21 Industrial), \n
Block 303 Lot 3 \n
(R-3 Single Family \n
Residential)HVAC  \n
Original / Various Ages\n
ROOFTOP SOLAR  \n
445.9 kW system installed 2005BUILDING COVERAGE  \n
39.2%\n
BLOCK / LOTS  \n
Morris Township: 10401/2 (97% of Property), \n
Morristown: 303/3 (3% of Property)  \n
OFFICE FINISH  \n
11,000 SF (3.7% office finish)\n
PROPERTY DESCRIPTION\n
Morris County  \n
Education\n
8,000 SF \n
8/31/22 LXD\n
Total Assessed: $14,236,100\n
Total 2021 Taxes: $273,685\n
Total PSF: $0.92REAL ESTATE TAXES\n
04 1 CORY ROAD, MORRISTOWN, NJ'
&nbsp;
[CORRECT VALUES FOR JSON KEYS]:
{
"Gross Building Area (GBA) - Square Feet": "296,000 SF",
"Year Built": "1986",
"Land Area (Acres)": "M.",
"Zoning": "Block 10401 Lot 2 (I-21 Industrial), Block 303 Lot 3 (R-3 Single Family",
"BLOCK / LOTS": "Morris Township: 10401/2 (97% of Property), Morristown: 303/3 (3% of Property)",
"Parking": "208 car, 80 Truck/Bus on +6.5 acres"
}
&nbsp;
"""


def generate_gpt_signature(email, instruct, examples):
    input_prompt = instruct + "\n" + email + "\n" + examples + "\n" + "[ANSWER]: "
    input_message = [{"role": "user", "content": input_prompt}]

    # get the response
    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=input_message,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    return response


def clean_pdf_text(text):
    return remove_citations(text)


def remove_citations(text):

    text = re.sub(r'\([A-Za-z0-9,.\s]+\s\d{4}\)', '', text)
    text = re.sub(r'\[[0-9,-]+(,\s[0-9,-]+)*\]', '', text)
    return text


def extract_text_from_pdf(file_path, page_numbers):
    all_text = ""
    pdf_reader = pypdf.PdfReader(file_path)
    
    for i, page in enumerate(pdf_reader.pages):
        if i + 1 == page_numbers:
            file_content = page.extract_text()
            if file_content:
                file_content = clean_pdf_text(file_content)
                all_text += file_content
            
    return all_text

def process_pdf(pdf_path, pages, instruct, examples):
    extracted_keywords = []

    for page in pages:
        extracted_text = extract_text_from_pdf(pdf_path, page)
        extracted_keywords.append(extracted_text)

    return extracted_keywords

# Test the function
pdf_path = '1-cory-rd-om-pdf/1.pdf'
pages = [1] 

response = process_pdf(pdf_path, pages, instruct=instruct, examples=examples)


In [2]:
with open('1-cory-rd-om_page_4.json', 'w') as jsonfile:
    json.dump(response, jsonfile)

[]

In [10]:
response

['THE OFFERING\nAs exclusive agent, Cushman & Wakefield’s National \nIndustrial Advisory Group is pleased to offer a 100% \nfee interest in 1 Cory Road for sale. Located less than \na mile from downtown Morristown, this 296,000 \nSF building on +/-18 acres represents the rare \nopportunity to an acquire infill industrial building \nwith an opportunistic investment profile.  \nThe property features 25 loading positions, 20’ \nclear heights, and is fully leased to four tenants at \nan average in-place rent of $4.44 PSF, a fraction \nof today’s $10.75 PSF market rate. The buildings \nlargest tenant (50% of GLA) rolls within 20 months, \npresenting the near-term ability to mark-to-market \nor reposition the building for high throughput use. \nThe property includes +/- 6.5 acres for parking \nwhich is currently underutilized and under-rented.  Situated less than five minutes from both Exit 36 on \nI-287 and Exit 1B on Route 24, the Property offers \nproximate access to NYC, the Port of NY/N